In [1]:
import tensorflow as tf

2024-03-29 16:36:47.501763: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-29 16:36:47.765780: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 16:36:47.765927: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 16:36:47.804723: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-29 16:36:47.894077: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Create an SCNet model using tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, Dropout, BatchNormalization, AvgPool1D, ReLU, Softmax
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAvgPool1D, Concatenate
from tensorflow.keras.optimizers import Adam

class TransposeLayer(tf.keras.layers.Layer):
    def __init__(self, perm=None, **kwargs):
        super(TransposeLayer, self).__init__(**kwargs)
        self.perm = perm

    def call(self, inputs):
        return tf.transpose(inputs, perm=self.perm)

def mffm_block(input):
    # Convolutional layer with ReLU activation
    x1 = Conv1D(filters=8, kernel_size=5, padding='same')(input)
    x1 = BatchNormalization()(x1)
    x1 = ReLU()(x1)

    # Concatenate input and x1 along the channel axis
    x2 = Concatenate(axis=-1)([input, x1])
    x2 = Conv1D(filters=16, kernel_size=5, padding='same')(x2)
    x2 = BatchNormalization()(x2)
    x2 = ReLU()(x2)

    # Concatenate input, x1, and x2 along the channel axis
    x3 = Concatenate(axis=-1)([input, x1, x2])
    return x3


def scnet(input_shape):
    input = Input(shape=input_shape)
    # Tranpose input
    x = TransposeLayer(perm=[0, 2, 1])(input)
    # Implementing SILM
    gap = GlobalAvgPool1D(data_format='channels_first',keepdims=True)(x)
    gmp = GlobalMaxPool1D(data_format='channels_first',keepdims=True)(x)
    gsp = tf.expand_dims(tf.sqrt(tf.reduce_sum((x - gap) ** 2, axis=2)) / 21, 2)
    # Apply dropout on each
    gap = Dropout(0.05)(gap)
    gmp = Dropout(0.05)(gmp)
    gsp = Dropout(0.05)(gsp)
    # Concatenate with input
    x = Concatenate(axis=2)([x, gap, gmp, gsp])

    # Take avg and maxpool
    x1 = MaxPool1D(3, strides = 3)(x)
    x2 = AvgPool1D(3, strides = 3)(x)
    x = Concatenate(axis=1)([x1, x2])
    x = BatchNormalization()(x)
    
    x1 = mffm_block(x)
    x2 = mffm_block(x)
    x = x1 + x2
    
    # Apply spatial dropout
    x = Dropout(0.5)(x)
    x = MaxPool1D(2, strides = 2)(x)
    x = Conv1D(32, 3)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x1 = mffm_block(x)
    x2 = mffm_block(x)
    x = x1 + x2

    x = Conv1D(32, 3)(x)
    x = BatchNormalization()(x)
    x = mffm_block(x)
    x = MaxPool1D(2, strides = 2)(x)
    x = Conv1D(32, 3)(x)
    x = BatchNormalization()(x)

    # Apply gap
    x = GlobalAvgPool1D(data_format='channels_first',keepdims=True)(x)

    # Flatten the output
    x = Flatten()(x)
    x = Dense(2)(x)

    # Apply softmax
    outputs = Softmax()(x)
    return Model(inputs=input, outputs=outputs)


model = scnet((21, 42001))

model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 21, 42001)]          0         []                            
                                                                                                  
 transpose_layer (Transpose  (None, 42001, 21)            0         ['input_1[0][0]']             
 Layer)                                                                                           
                                                                                                  
 global_average_pooling1d (  (None, 42001, 1)             0         ['transpose_layer[0][0]']     
 GlobalAveragePooling1D)                                                                          
                                                                                              

In [4]:
from Arsalan.tuh import TUHDataset
dataset = TUHDataset('/home/athar/Files/TUKL_work')

In [5]:
train = dataset.create_dataset(1)
train

<_FlatMapDataset element_spec=(TensorSpec(shape=(21, 42001), dtype=tf.float32, name=None), TensorSpec(shape=(2,), dtype=tf.bool, name=None))>

In [8]:
model.fit(train.repeat().padded_batch(32).take(10), epochs=1)

2024-03-29 16:38:09.458260: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-29 16:38:09.486480: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 16:38:09.486520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 16:38:09.487228: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-29 16:38:09.492583: I tensorflow/core/platform/cpu_feature_guar

In [30]:
model.save('scnet.keras')

In [ ]:
model = tf.keras.models.load_model('scnet.keras')